In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import copy

# Kiểm tra xem có GPU không, nếu không thì dùng CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Model(nn.Module):
    def __init__(self, backbone='resnet18', n_classes=2):
        super(Model, self).__init__()
        if backbone == 'resnet18':
            self.model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Linear(num_ftrs, n_classes)
        elif backbone == 'mobilenet_v2':
            self.model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
            num_ftrs = self.model.classifier[1].in_features
            self.model.classifier[1] = nn.Linear(num_ftrs, n_classes)
        else:
            raise ValueError("Unsupported backbone")

    def forward(self, x):
        return self.model(x)


class Learner:
    def __init__(self, model, train_dataloader, val_dataloader, criterion, optimizer):
        self.model = model.to(device)
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.criterion = criterion
        self.optimizer = optimizer
        self.best_model_wts = copy.deepcopy(self.model.state_dict())
        self.best_acc = 0.0

    def train_epoch(self):
        self.model.train()
        running_loss = 0.0
        corrects = 0

        for inputs, labels in self.train_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels % 2  # Đảm bảo nhãn trong khoảng hợp lệ
            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(self.train_dataloader.dataset)
        epoch_acc = corrects.double() / len(self.train_dataloader.dataset)
        return epoch_loss, epoch_acc.item()

    def val_epoch(self):
        self.model.eval()
        running_loss = 0.0
        corrects = 0

        with torch.no_grad():
            for inputs, labels in self.val_dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                labels = labels % 2  # Đảm bảo nhãn trong khoảng hợp lệ
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(self.val_dataloader.dataset)
        epoch_acc = corrects.double() / len(self.val_dataloader.dataset)

        if epoch_acc > self.best_acc:
            self.best_acc = epoch_acc
            self.best_model_wts = copy.deepcopy(self.model.state_dict())

        return epoch_loss, epoch_acc.item()

    def train(self, num_epochs=25):
        for epoch in range(num_epochs):
            train_loss, train_acc = self.train_epoch()
            val_loss, val_acc = self.val_epoch()
            print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

        self.model.load_state_dict(self.best_model_wts)
        return self.model

    def test(self, test_dataloader):
        self.model.eval()
        running_loss = 0.0
        corrects = 0

        with torch.no_grad():
            for inputs, labels in test_dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                labels = labels % 2  # Đảm bảo nhãn trong khoảng hợp lệ
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                corrects += torch.sum(preds == labels.data)

        loss_mean = running_loss / len(test_dataloader.dataset)
        acc_mean = corrects.double() / len(test_dataloader.dataset)
        return acc_mean.item(), loss_mean

    def inference(self, image_tensor):
        self.model.eval()
        with torch.no_grad():
            image_tensor = image_tensor.to(device).unsqueeze(0)
            output = self.model(image_tensor)
            _, pred = torch.max(output, 1)
        return pred.item()

In [12]:
def main():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    dataset = datasets.FakeData(transform=transform)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model = Model(backbone='resnet18', n_classes=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    learner = Learner(model, train_loader, val_loader, criterion, optimizer)
    trained_model = learner.train(num_epochs=10)

if __name__ == "__main__":
    main()

Epoch 1/10 - Train Loss: 0.7761, Acc: 0.5088 | Val Loss: 0.8354, Acc: 0.4750
Epoch 2/10 - Train Loss: 0.6045, Acc: 0.6562 | Val Loss: 1.6802, Acc: 0.5250
Epoch 3/10 - Train Loss: 0.3153, Acc: 0.8588 | Val Loss: 1.2399, Acc: 0.5250
Epoch 4/10 - Train Loss: 0.2035, Acc: 0.9213 | Val Loss: 2.9996, Acc: 0.5150
Epoch 5/10 - Train Loss: 0.0877, Acc: 0.9600 | Val Loss: 5.1914, Acc: 0.4750
Epoch 6/10 - Train Loss: 0.1267, Acc: 0.9563 | Val Loss: 1.6104, Acc: 0.4900
Epoch 7/10 - Train Loss: 0.0814, Acc: 0.9625 | Val Loss: 1.5847, Acc: 0.4800
Epoch 8/10 - Train Loss: 0.0744, Acc: 0.9663 | Val Loss: 4.4306, Acc: 0.4750
Epoch 9/10 - Train Loss: 0.0734, Acc: 0.9775 | Val Loss: 1.5950, Acc: 0.4950
Epoch 10/10 - Train Loss: 0.0670, Acc: 0.9700 | Val Loss: 1.5624, Acc: 0.5350
